In [1]:
import zipfile
from glob import glob
from tifffile import imread
#from cellpose import models #in case models are nto downloaded yet

# DISPLAY RESULTS imports
#from cellpose import plot
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os
from scipy.ndimage import zoom
import skimage.io
from PIL import Image
import pandas as pd
import torch
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import torchvision
import torchvision.transforms as T
import numpy as np
from torchvision.models.segmentation.fcn import FCNHead
from torchvision.models.segmentation.deeplabv3 import DeepLabHead
import time
import os
import pandas as pd
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from glob import glob
import argparse
import torch
import torchvision
from PIL import Image
import torch.nn as nn
import torchvision.transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from scipy.ndimage import morphology
sys.path.append(os.path.join(sys.path[0]))  # To find local version of the library

from torchvision.models.detection.rpn import AnchorGenerator
import torch
from skimage.color import rgb2gray, label2rgb
from skimage import io,exposure, feature, filters, io, measure, morphology, restoration, segmentation, transform, util, data, color
from skimage.measure import label, regionprops
from skimage.transform import rescale, resize, downscale_local_mean
import os
import numpy as np
import os
import torch.nn as nn
#load image
from glob import glob
import numpy as np
from skimage.morphology import remove_small_objects,dilation, erosion, ball  
from skimage import (exposure, feature, filters, io, measure,
                      morphology, restoration, segmentation, transform,
                      util)
import matplotlib.pyplot as plt
import skimage
from scipy.ndimage import zoom
import copy
import pyclesperanto_prototype as cle #GPU Image anlaysis 
import os
import scipy.ndimage
from skimage import measure, morphology
import gc
from skimage.util import invert
from skimage.morphology import remove_small_objects,dilation, erosion, ball  
from skimage import (exposure, feature, filters, io, measure,
                      morphology, restoration, segmentation, transform,
                      util)
import matplotlib.pyplot as plt
import skimage
from scipy.ndimage import zoom
import copy
import pyclesperanto_prototype as cle #GPU Image anlaysis 
import os
import scipy.ndimage
from skimage import measure, morphology
import gc
from skimage.util import invert
import pandas as pd
from vedo import *
import pyvista as pv
import porespy as ps
import openpnm as op
import scipy.ndimage as spim
from porespy.filters import find_peaks, trim_saddle_points, trim_nearby_peaks
from porespy.tools import randomize_colors
from skimage.segmentation import watershed
from skimage.transform import rescale



class Material:
 
  def __init__(self, name, input_rgb_vals, output_val, confidence_threshold=0):
    self.name = name
    self.input_rgb_vals = input_rgb_vals
    self.output_val = output_val
    self.confidence_threshold = confidence_threshold

In [2]:
# Models 50 epochs of just the tubes 30%
#model_group='ansto_ai_scaled/'
#current_model_name = 'dec152023_soil_ai_30perc50epoch'
#model_number="1"


# model mixed (sml amound of croppped)
#model_group = 'ansto_ai_MIXED/'
#current_model_name = 'mixed'

#as above but 50 epochs
#model_group='mixed_training_data/'
#current_model_name = 'mixed_training_data_sml_50epoch'

#as above but all data  4.5594e-13
#model_group='mixed_training_data_ALL/'
#current_model_name = 'mixed_training_data_ALL_50epoch'

#
#model_group='model_with_reslice_test\\'
#current_model_name = 'model_with_reslice_100epochs'

In [3]:
#root or not root
#Creating a list of materials so we can iterate through it
materials = [
            Material("plastic", [1,1,1],  30, 0.8),
            Material("root", [2,2,2], 175, 0.2),
            Material("soil", [3,3,3], 100, 0.4),
            Material("pore", [4,4,4], 255, 0.6),
            Material("background", [5,5,5], 1, 0.5)]
 
num_materials =len(materials)
#Decrease scale to decrease VRAM usage; if you run out of VRAM during traing, restartyour runtime and down scale your images
scale=1
#Input model directory
models_directory = 'F:\\UCT_AI\\FNN\\best_models\\'
#Input the name you want to use for your group of models
model_group='devin_r\\'
current_model_name = '100_epoch'
model_number="3"


###############################
#Code Box 9
model=torchvision.models.segmentation.fcn_resnet101(pretrained=False)
model.classifier=FCNHead(2048, num_materials)
device = torch.device('cuda')
outputs=[]
model.to(device)
#!!!!!!!!!!!!!!!!!!!!!Select Correct Model from the best models directory!!!!!!!!!!!!!!!!!!!!!!!!!1
model.load_state_dict(torch.load(models_directory+model_group + current_model_name+model_number+'.pth'), strict=False)
#model.train()
mean=-3.3307e-16 #This is silly just copy and paste value from model train - this should be saved as a text file ?
std=1


In [4]:
#Part 2 Segment using AI (Segment)
#Loop through folders#######################################
ct_folder_path_name ='G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\'
#ct_folder_path_name ='F:\\UCT_AI\\8_bit_test\\'

ct_folder_paths=np.array((glob(ct_folder_path_name+"*\\", recursive = True)))
########
#ct_folder_paths = ct_folder_paths[ct_folder_paths == 'F:\\UCT_AI\\\\RAW_ANSTO_DATA_AS_STACKS\\rabbi_data_I_think\\']
print (ct_folder_paths)
############################

['G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_BALDY_D_S1_R1.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_BALDY_D_S1_R3.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_BALDY_WW_S1_R1.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_BALDY_WW_S1_R2.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_BALDY_WW_S1_R3.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_BARLEYWT_D_S1_R2.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_BARLEYWT_D_S1_R3.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_BARLEYWT_WW_S1_R1.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_BARLEYWT_WW_S1_R2.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_BARLEYWT_WW_S1_R3.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_GALVEZ_TALL_S1_R1.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_GALVEZ_TALL_S1_R2.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_KAUZ_DWARF_S1_R2.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_A

In [5]:
#edit ct paths 
ct_folder_paths=ct_folder_paths[16:]
print (ct_folder_paths)

['G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_NESSER_DWARF_S1_R3.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_PAVON_DWARF_S1_R1.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_PAVON_DWARF_S1_R2.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_PAVON_DWARF_S1_R3.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_SERI_DWARF_S1_R1.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_SERI_TALL_S1_R2.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_SERI_TALL_S1_R3_v2.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_W16_D_S1_R2.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_W16_D_S1_R3.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_W16_WW_S1_R3.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_W1_D_S1_R1.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_W1_WW_S1_R1_V4.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\recon_W1_WW_S1_R2.tif\\'
 'G:\\SOIL_UCT\\RAW_ANSTO_DATA_AS_XY_STACKS\\r

In [6]:
#########################################
output_base = "G:\\SOIL_UCT\\SEGMENTED_OUT\\"
for folder_x in ct_folder_paths:
    #print (folder_x)
    IMAGE_ID = folder_x.replace(ct_folder_path_name , "")
    IMAGE_ID = IMAGE_ID.replace('\\', '')
    print (IMAGE_ID)
    output_directory = output_base+IMAGE_ID+"\\"
    if os.path.isdir(output_directory):
        out=(output_directory)
    else:
        out=os.mkdir(output_directory)
        
    inference_directory= folder_x
    proceeding="lice_"
    following=".tif"
    dir_name = inference_directory
    filenames = os.listdir(dir_name)
    sort_idx = np.argsort([(int(filename.split(proceeding)[1].split(following)[0])) for filename in filenames])
    for i in sort_idx:
        #makes new directory called "(directory name here) + name in red" that your new images go into
        new_dir_name = output_directory
        if not os.path.exists(new_dir_name):
          os.makedirs(new_dir_name)

        for mat in materials:
          new_dir_name_mat= new_dir_name + mat.name
          if not os.path.exists(new_dir_name_mat):
            os.makedirs(new_dir_name_mat)
        filename = filenames[i]

        image = Image.open(dir_name +'/'+ filenames[i])
        image1 = (image)
        image1 = np.array(image1, dtype='ubyte')
        zeros=np.zeros_like(image1)

        w, h = image.size
        #print(image.size)
        #!!!!!!!!!!!!!!!!!!!!Make sure scale matches!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        scale=scale
        newW, newH = int(scale * w), int(scale * h)
        image=image.resize((newW, newH))
        image_resize=image
        image=np.array(image)
        zeros2=np.zeros_like(image)
        new_im=np.zeros((3, newH, newW))
        new_im[0,:,:]=image
        new_im[1,:,:]=image
        new_im[2,:,:]=image
        image=new_im
        image=torch.from_numpy(image)
        image=T.Normalize(mean=[mean, mean , mean], std=[std, std , std])(image)
        image.unsqueeze_(0)
        image = image.to(device=device, dtype=torch.float32)
        tic=time.time()
        with torch.no_grad():
          mask=model(image)['out']
          mask=nn.Sigmoid()(mask)
        toc=time.time()
        #print('time: '+str(toc-tic))

        image_rescaled=zeros2
        combined_image = np.array(image_rescaled, dtype='ubyte') 

        list_of_mat_tables = []
        for i, mat in enumerate(materials):
          mat_mask = mask.cpu().detach().numpy()[0,i,:,:]
          mat_mask[mat_mask >= mat.confidence_threshold] = mat.output_val
          mat_mask[mat_mask < mat.confidence_threshold] = 0
          mat_mask=np.array(mat_mask, dtype='ubyte')
          combined_image = np.add(combined_image, mat_mask[:,:])

          io.imsave(new_dir_name+'/' + mat.name + '/'+filename.split(following)[0]+'_' + mat.name + "_mask.png",
                    mat_mask, check_contrast=False)
          np_mat = np.array(mat_mask)


        io.imsave(new_dir_name+'/'+filename.split(following)[0]+'.png', combined_image, check_contrast=False)   
        
        



recon_NESSER_DWARF_S1_R3.tif
recon_PAVON_DWARF_S1_R1.tif
recon_PAVON_DWARF_S1_R2.tif
recon_PAVON_DWARF_S1_R3.tif
recon_SERI_DWARF_S1_R1.tif
recon_SERI_TALL_S1_R2.tif
recon_SERI_TALL_S1_R3_v2.tif
recon_W16_D_S1_R2.tif
recon_W16_D_S1_R3.tif
recon_W16_WW_S1_R3.tif
recon_W1_D_S1_R1.tif
recon_W1_WW_S1_R1_V4.tif
recon_W1_WW_S1_R2.tif
recon_W1_WW_S1_R3.tif
